Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        new_orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        current_time = data.get('utc')
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price, current_time)
            new_orders.append(order)
        elif ctx.record.balance.get(self.asset.base)[BalanceType.FREE] > quantity:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity, current_time)
            new_orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        self.log_all(new_orders, data, ctx, current_time)
        return {
            'new_orders': new_orders,
            'cancel_ids': cancel_ids
        }

## Backtest

### Backtest From Local CSV Files

In [3]:
# Download some data from Binance
exchange_id = ex_cfg.BINANCE
timeframe = Timeframe.THIRTY_MIN
start = datetime.datetime(year=2018, month=1, day=1, hour=0)
end = datetime.datetime(year=2018, month=1, day=2, hour=0)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.BTC, coins.USDT)]
data_exchange = load_exchange(exchange_id)
ohlcv_feed.download_ohlcv([data_exchange], assets, timeframe, start, end)

Downloading: ETH/BTC
Downloaded rows: 48
Downloading: BTC/USDT
Downloaded rows: 48


In [ ]:
# Setup the Feed Based Paper Exchange
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=timeframe
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=deepcopy(balance),
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    exchange_ids=[exchange_id],
    assets=assets,
    timeframe=timeframe,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(deepcopy(balance), feed, exchange_id)
strategy = SimpleStrategy(assets[0], quantity=.05)
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

In [ ]:
record

### Backtest from External Exchange Data

In [ ]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=balance,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [4]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=balance,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

Downloading: ETH/BTC
Downloaded rows: 0
Downloading: LTC/BTC
Downloaded rows: 0
Downloading: BTC/USDT
Downloaded rows: 0


In [7]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 2
Downloading: LTC/BTC


---------------------------------------
Epoch 2 - Timestep: 2018-01-27T21:38:00
---------------------------------------
OHLCV
    O: 0.0967 | C: 0.0968 | V: 17.3 | T: 2018-01-27T21:38:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0968 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 3 - Timestep: 2018-01-27T21:38:00
---------------------------------------
OHLCV
    O: 0.0967 | C: 0.0968 | V: 17.3 | T: 2018-01-27T21:38:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0968 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9952 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99516145, 'used': 0.0, 'total': 0.99516145}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, '

Downloaded rows: 2
Downloading: BTC/USDT
Downloaded rows: 2
TIME 2018-01-27 21:38:25.614643
Inspecting trade times...
trade time 2018-01-27 21:38:25.614940
last_update_time 2018-01-27 21:37:30
TIME 2018-01-27 21:38:25.630025
Inspecting trade times...
trade time 2018-01-27 21:38:25.630357
last_update_time 2018-01-27 21:38:25.619017


    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9903 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99032290000000001, 'used': 0.0, 'total': 0.99032290000000001}
    ETH - {'free': 0.1, 'used': 0.0, 'total': 0.1}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.1, 'cost_price': 0.09677100000000001, 'latest_price': 0.09677100000000001, 'fee': 0.0}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 5 - Timestep: 2018-01-27T21:38:00
---------------------------------------
OHLCV
    O: 0.0967 | C: 0.0968 | V: 17.3 | T: 2018-01-27T21:38:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0968 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9952 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99516154999999995, 'used': 0.0, 'total': 0.99516154999999995}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.

TIME 2018-01-27 21:38:25.781831
Inspecting trade times...
trade time 2018-01-27 21:38:25.782541
last_update_time 2018-01-27 21:38:25.634362
TIME 2018-01-27 21:38:25.794524
Inspecting trade times...
trade time 2018-01-27 21:38:25.794791
last_update_time 2018-01-27 21:38:25.785887
TIME 2018-01-27 21:38:25.811564
Inspecting trade times...
trade time 2018-01-27 21:38:25.811905
last_update_time 2018-01-27 21:38:25.799931
TIME 2018-01-27 21:38:25.958818
Inspecting trade times...
trade time 2018-01-27 21:38:25.959205
last_update_time 2018-01-27 21:38:25.817464


...

---------------------------------------
Epoch 338 - Timestep: 2018-01-27T21:38:00
---------------------------------------
OHLCV
    O: 0.0967 | C: 0.0968 | V: 17.3 | T: 2018-01-27T21:38:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.8743 Total Val: 1.0001 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 0.87425200000000447, 'used': 0.0, 'total': 0.87425200000000447}
    ETH - {'free': 1.3000000000000005, 'used': 0.0, 'total': 1.3000000000000005}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1.3000000000000005, 'cost_price': 0.096770999999999857, 'latest_price': 0.09677100000000001, 'fee': 0.0}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 339 - Timestep: 2018-01-27T21:38:00
---------------------------------------
OHLCV
    O: 0.0967 | C: 0.0968 | V: 17.3 | T: 2018-01-27T21:38:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.8791 Total Val: 1.0

TIME 2018-01-27 21:38:59.655533
Inspecting trade times...
trade time 2018-01-27 21:38:59.656082
last_update_time 2018-01-27 21:38:59.513707
TIME 2018-01-27 21:38:59.848064
Inspecting trade times...
trade time 2018-01-27 21:38:59.848459
last_update_time 2018-01-27 21:38:59.709159


---------------------------------------
Epoch 340 - Timestep: 2018-01-27T21:38:00
---------------------------------------
OHLCV
    O: 0.0967 | C: 0.0968 | V: 17.3 | T: 2018-01-27T21:38:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.8839 Total Val: 1.0001 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 0.88393000000000455, 'used': 0.0, 'total': 0.88393000000000455}
    ETH - {'free': 1.2000000000000004, 'used': 0.0, 'total': 1.2000000000000004}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1.2000000000000004, 'cost_price': 0.096770999999999857, 'latest_price': 0.09677100000000001, 'fee': 0.0}
METRICS
    SMA: 5.00
    RSI: 10.00


TIME 2018-01-27 21:39:00.045177
Inspecting trade times...
trade time 2018-01-27 21:39:00.045734
last_update_time 2018-01-27 21:38:59.900901
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-27 21:38:00


AttributeError: 'NoneType' object has no attribute 'get'

## Live Trade

In [ ]:
exchange = load_exchange(ex_cfg.BINANCE)
cash = exchange.fetch_balance().get(coins.BTC)[BalanceType.FREE]
cash_currency = coins.BTC
print(coins.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=None,
    end=None
)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)